In [1]:
import pandas as pd

In [2]:
def map_income(income):
    if income<=9036.8:
        return 'Low'
    else:
        return 'High'

In [3]:
average_number = pd.read_csv('../../TABLAS LATEX/languages.csv', sep=';', decimal=',')
average_number = average_number[['Language', 'Average']]
average_number

,Language,Average
0,English,5607.00
1,Spanish,2348.33
2,French,832.33
3,Chinese,770.00
4,Italian,579.00
5,Russian,560.33
6,Portuguese,462.67
7,Arabic,418.67
8,Japanese,410.33
9,German,388.67


# LOAD BIG MAC INDEX

In [4]:
big_mac_index = pd.read_csv('../data/big-mac-2021-01-01.csv')

In [5]:
big_mac_index = big_mac_index[['iso_a3', 'dollar_price']]
big_mac_index = big_mac_index.rename(columns={'dollar_price': 'big_mac_dollar_price', 'iso_a3': 'nationality'})
big_mac_index.head()

,nationality,big_mac_dollar_price
0,ARG,3.748231
1,AUS,4.984740
2,AZE,2.324897
3,BHR,3.978780
4,BRA,3.978491


In [6]:
import pycountry

stored_countries = {}

def map_iso2_to_iso3(iso2):
    try:
        if iso2 not in stored_countries:
            iso3 = pycountry.countries.get(alpha_2=iso2).alpha_3
            stored_countries[iso2] = iso3
            return iso3
        else:
            return stored_countries[iso2]
    except:
        print(iso2)
        return None

# Preply

In [91]:
preply = pd.read_csv('../data/results/final_dataframes/preply.csv', index_col=0)
preply = preply.drop_duplicates(subset=['user_name', 'language'])

In [92]:
preply['income_level'] = preply['income_level'].apply(map_income)

In [93]:
preply['nationality'] = preply['nationality'].apply(map_iso2_to_iso3)

nan
nan
nan
nan
nan
nan


In [94]:
preply = pd.merge(preply, big_mac_index, on='nationality', how='left')

In [ ]:
preply.head()

In [ ]:
preply['price_to_big_mac'] = preply.price/preply.big_mac_dollar_price
preply.head()

In [97]:
preply.columns

Index(['language', 'position', 'retrieval_date', 'is_featured', 'user_name',
       'url', 'nationality_full', 'avg_rating', 'num_ratings', 'teaches',
       'subjects', 'speaks', 'lessons', 'price', 'price_currency',
       'avatar_url', 'nationality', 'clean_name', 'sanitized_name', 'gender',
       'probability', 'count', 'first_name', 'probability_male',
       'probability_female', 'gender_tuned', 'income_level', 'Code',
       'big_mac_dollar_price', 'price_to_big_mac'],
      dtype='object')

In [98]:
from scipy.stats import ks_2samp
import numpy as np

def hypothesis_test(group1, group2, alpha=0.1):
    st, p_value = ks_2samp(group1, group2)
    if p_value<alpha:
        return st, p_value
    else:
        return st, p_value
    
def compute_aggregated_feature_top_k(df, top_k, language_col, aggregation_col, target_cols, group1, group2):
    count_group1 = 'count_{}'.format(group1)
    count_group2 = 'count_{}'.format(group2)
    variance_group1 = 'variance_{}'.format(group1)
    variance_group2 = 'variance_{}'.format(group2)
    mean_group1 = 'mean_{}'.format(group1)
    mean_group2 = 'mean_{}'.format(group2)
    median_group1 = 'median_{}'.format(group1)
    median_group2 = 'median_{}'.format(group2)
    
    results = pd.DataFrame(columns=['language', 'top_k', 'target_col', 'aggregation_col', mean_group1, median_group1, mean_group2,  median_group2, count_group1, count_group2, variance_group1, variance_group2, 'statistic', 'p_value'])
    
    for lang in df[language_col].unique():
        temp = df[df[language_col]==lang]
        temp = temp.sort_values(by='position', ascending=True)
        
        for target in target_cols:
            temp = temp.dropna(subset=[target])
            if top_k is not None:
                temp = temp.head(top_k)
            
            temp[target] = pd.to_numeric(temp[target], errors='coerce')
            g1 = temp[temp[aggregation_col]==group1][target].values
            g2 = temp[temp[aggregation_col]==group2][target].values
            
            g1_count = len(g1)
            g2_count = len(g2)

            g1_mean = np.nanmean(g1) if g1_count else None
            g2_mean = np.nanmean(g2) if g2_count else None
            
            g1_median = np.nanmedian(g1) if g1_count else None
            g2_median = np.nanmedian(g2) if g2_count else None
            
            g1_var = g1.var() if g1_count else None
            g2_var = g2.var() if g2_count else None
            
            #Hypothesis testing
            if len(g1)>0 and len(g2)>0:
                st, p_value = hypothesis_test(g1, g2)
            else:
                st, p_value = None, None
            
            results = results.append({'language': lang, 'top_k': len(temp), 'target_col': target, 'aggregation_col': aggregation_col,
                                      mean_group1: g1_mean, median_group1: g1_median, mean_group2: g2_mean, median_group2: g2_median, count_group1: g1_count, count_group2: g2_count,
                                      variance_group1: g1_var, variance_group2: g2_var, 'statistic': st, 'p_value': p_value}, ignore_index=True)

    return results

In [99]:
preply_results_40 = compute_aggregated_feature_top_k(preply, 40, 'language', 'income_level', ['price_to_big_mac'], 'High', 'Low')

In [100]:
preply_results_all = compute_aggregated_feature_top_k(preply, None, 'language', 'income_level', ['price_to_big_mac'], 'High', 'Low')

In [101]:
preply_results = pd.concat([preply_results_40, preply_results_all])

In [102]:
preply_results = pd.merge(preply_results, average_number, how='left', left_on='language', right_on='Language')
preply_results = preply_results.rename(columns={'Average': 'average_num_teachers'})

In [103]:
preply_results.head()

,language,top_k,target_col,aggregation_col,mean_High,median_High,mean_Low,median_Low,count_High,count_Low,variance_High,variance_Low,statistic,p_value,Language,average_num_teachers
0,Thai,40,price_to_big_mac,income_level,2.29682,2.29682,3.14458,3.06008,1,39,0,0.541264,0.871795,0.3,Thai,51.67
1,German,40,price_to_big_mac,income_level,6.94019,6.39409,9.14018,7.49497,28,12,13.0541,44.437,0.321429,0.291241,German,388.67
2,Portuguese,40,price_to_big_mac,income_level,6.35737,6.35737,3.25434,3.26757,2,38,0.687609,3.1188,0.921053,0.025641,Portuguese,462.67
3,Hebrew,40,price_to_big_mac,income_level,5.98853,4.73675,7.60538,8.62355,37,3,6.68544,3.46873,0.585586,0.209717,Hebrew,46.67
4,Hindi,40,price_to_big_mac,income_level,None,None,4.65484,4.24889,0,40,None,3.92184,None,None,Hindi,78.00


In [104]:
preply_results.to_csv('../data/results/features_analysis/income/preply_big_mac.csv')

# Italki

In [105]:
italki = pd.read_csv('../data/results/final_dataframes/italki.csv', index_col=0)
italki = italki.drop_duplicates(subset=['user_id', 'language'])

In [106]:
italki['income_level'] = italki['income_level'].apply(map_income)

In [107]:
italki['nationality'] = italki['nationality'].apply(map_iso2_to_iso3)

ZZ
ZZ
ZZ
ZZ
ZZ


In [108]:
italki = pd.merge(italki, big_mac_index, on='nationality', how='left')

In [ ]:
italki.head()

In [ ]:
italki['price_to_big_mac'] = italki.price/italki.big_mac_dollar_price/100
italki.head()

In [ ]:
italki.head()

In [112]:
italki.columns

Index(['position', 'retrieval_date', 'user_id', 'user_name',
       'avatar_file_name', 'video_picture', 'is_pro', 'nationality', 'teaches',
       'also_speaks', 'in_platform_since', 'rating', 'number_sessions',
       'price', 'price_time', 'price_currency', 'clean_name', 'sanitized_name',
       'gender', 'probability', 'count', 'language', 'probability_male',
       'probability_female', 'gender_tuned', 'income_level', 'Code',
       'big_mac_dollar_price', 'price_to_big_mac'],
      dtype='object')

In [113]:
italki_results_40 = compute_aggregated_feature_top_k(italki, 40, 'language', 'income_level', ['price_to_big_mac'], 'High', 'Low')

In [114]:
italki_results_all = compute_aggregated_feature_top_k(italki, None, 'language', 'income_level', ['price_to_big_mac'], 'High', 'Low')

In [115]:
italki_results = pd.concat([italki_results_40, italki_results_all])

In [116]:
italki_results = pd.merge(italki_results, average_number, how='left', left_on='language', right_on='Language')
italki_results = italki_results.rename(columns={'Average': 'average_num_teachers'})

In [117]:
italki_results.head()

,language,top_k,target_col,aggregation_col,mean_High,median_High,mean_Low,median_Low,count_High,count_Low,variance_High,variance_Low,statistic,p_value,Language,average_num_teachers
0,Thai,40,price_to_big_mac,income_level,5.20321,5.20321,2.42242,2.35391,1,39,0,0.450734,1,0.05,Thai,51.67
1,German,40,price_to_big_mac,income_level,4.83215,4.81028,5.86508,5.48323,30,10,3.93119,5.95515,0.333333,0.34324,German,388.67
2,Portuguese,40,price_to_big_mac,income_level,7.11422,7.11422,2.67132,2.51352,2,38,11.4871,1.39292,0.894737,0.0384615,Portuguese,462.67
3,Hebrew,40,price_to_big_mac,income_level,3.34411,3.0855,None,None,40,0,2.28493,None,None,None,Hebrew,46.67
4,Hindi,40,price_to_big_mac,income_level,None,None,2.94014,2.70384,0,40,None,1.20667,None,None,Hindi,78.00


In [118]:
italki_results.to_csv('../data/results/features_analysis/income/italki_big_mac.csv')

In [119]:
italki_results[italki_results['p_value']<0.1]

,language,top_k,target_col,aggregation_col,mean_High,median_High,mean_Low,median_Low,count_High,count_Low,variance_High,variance_Low,statistic,p_value,Language,average_num_teachers
0,Thai,40,price_to_big_mac,income_level,5.20321,5.20321,2.42242,2.35391,1,39,0,0.450734,1,0.05,Thai,51.67
2,Portuguese,40,price_to_big_mac,income_level,7.11422,7.11422,2.67132,2.51352,2,38,11.4871,1.39292,0.894737,0.0384615,Portuguese,462.67
10,Russian,40,price_to_big_mac,income_level,7.35031,6.63378,4.66749,4.53871,26,14,6.34375,2.80027,0.664835,0.000248062,Russian,560.33
18,Spanish,40,price_to_big_mac,income_level,3.61666,3.35246,2.62459,2.66953,21,19,2.42344,0.622399,0.413534,0.0451588,Spanish,2348.33
22,Thai,79,price_to_big_mac,income_level,4.29166,4.29166,2.51291,2.35391,2,77,0.830913,0.503792,0.87013,0.0428432,Thai,51.67
23,German,87,price_to_big_mac,income_level,4.47221,3.8697,5.38272,5.44645,67,20,6.14612,5.13202,0.317164,0.0695336,German,388.67
24,Portuguese,295,price_to_big_mac,income_level,3.83785,3.13946,2.6104,2.511,28,267,7.13996,1.36046,0.356474,0.00219987,Portuguese,462.67
32,Russian,337,price_to_big_mac,income_level,6.1209,5.52815,4.51009,4.53871,243,94,5.88484,2.76062,0.384642,1.72978e-09,Russian,560.33
34,English,298,price_to_big_mac,income_level,4.43909,3.97139,4.69316,4.17022,217,81,4.22481,5.61006,0.163509,0.0745704,English,5607.00
37,Turkish,141,price_to_big_mac,income_level,4.88263,4.48529,2.60066,2.3657,135,6,4.17963,0.648287,0.648148,0.00726338,Turkish,151.33


# Verbling

In [120]:
verbling = pd.read_csv('../data/results/final_dataframes/verbling.csv', index_col=0)
verbling = verbling.drop_duplicates(subset=['first_name', 'last_name', 'language'])

In [121]:
verbling['income_level'] = verbling['income_level'].apply(map_income)

In [122]:
verbling['nationality'] = verbling['nationality'].apply(map_iso2_to_iso3)

In [123]:
verbling = pd.merge(verbling, big_mac_index, on='nationality', how='left')

In [ ]:
verbling.head()

In [ ]:
verbling['price_to_big_mac'] = verbling.price/verbling.big_mac_dollar_price
verbling.head()

In [126]:
verbling.columns

Index(['language', 'position', 'retrieval_date', 'is_featured', 'first_name',
       'last_name', 'url', 'nationality', 'location', 'avg_rating',
       'avg_lessons_per_students', 'num_ratings', 'teaching_levels', 'teaches',
       'class_details', 'speaks', 'lessons', 'students', 'dialect',
       'price_currency', 'avatar_url', 'clean_name', 'sanitized_name',
       'gender', 'probability', 'count', 'price_detail', 'price',
       'probability_male', 'probability_female', 'gender_tuned',
       'income_level', 'Code', 'big_mac_dollar_price', 'price_to_big_mac'],
      dtype='object')

In [127]:
verbling_results_40 = compute_aggregated_feature_top_k(verbling, 40, 'language', 'income_level', ['price_to_big_mac'], 'High', 'Low')

In [128]:
verbling_results_all = compute_aggregated_feature_top_k(verbling, None, 'language', 'income_level', ['price_to_big_mac'], 'High', 'Low')

In [129]:
verbling_results = pd.concat([verbling_results_40, verbling_results_all])

In [130]:
verbling_results = pd.merge(verbling_results, average_number, how='left', left_on='language', right_on='Language')
verbling_results = verbling_results.rename(columns={'Average': 'average_num_teachers'})

In [131]:
verbling_results.head()

,language,top_k,target_col,aggregation_col,mean_High,median_High,mean_Low,median_Low,count_High,count_Low,variance_High,variance_Low,statistic,p_value,Language,average_num_teachers
0,Thai,18,price_to_big_mac,income_level,4.78392,4.50682,4.11149,4.00164,3,15,0.530161,0.39057,0.666667,0.161765,Thai,51.67
1,German,6,price_to_big_mac,income_level,7.52568,5.21727,None,None,6,0,28.927,None,None,None,German,388.67
2,Portuguese,40,price_to_big_mac,income_level,None,None,3.72622,3.51892,0,40,None,1.79723,None,None,Portuguese,462.67
3,Hebrew,9,price_to_big_mac,income_level,7.10434,6.545,None,None,9,0,2.16913,None,None,None,Hebrew,46.67
4,Hindi,15,price_to_big_mac,income_level,3.53357,3.53357,6.40093,5.79395,1,14,0,4.15017,0.928571,0.266667,Hindi,78.00


In [132]:
verbling_results.to_csv('../data/results/features_analysis/income/verbling_big_mac.csv')

In [133]:
verbling_results[verbling_results['p_value']<0.1]

,language,top_k,target_col,aggregation_col,mean_High,median_High,mean_Low,median_Low,count_High,count_Low,variance_High,variance_Low,statistic,p_value,Language,average_num_teachers
16,Vietnamese,20,price_to_big_mac,income_level,1.88457,2.12014,3.81317,3.844,3,17,0.110988,0.67968,1,0.00175439,Vietnamese,49.67
18,Japanese,40,price_to_big_mac,income_level,6.80777,6.41815,3.26757,3.26757,39,1,5.69497,0,0.974359,0.1,Japanese,410.33
32,Russian,82,price_to_big_mac,income_level,9.06237,8.29222,6.6129,6.35419,72,10,15.0462,4.80062,0.466667,0.0291209,Russian,560.33
34,English,410,price_to_big_mac,income_level,4.56371,4.24028,6.6529,6.33361,354,56,3.14594,5.36623,0.470137,2.93362e-10,English,5607.00
38,Vietnamese,20,price_to_big_mac,income_level,1.88457,2.12014,3.81317,3.844,3,17,0.110988,0.67968,1,0.00175439,Vietnamese,49.67
41,Spanish,256,price_to_big_mac,income_level,5.18866,5.21494,4.12457,4.0083,149,107,4.73647,2.41526,0.289845,3.99042e-05,Spanish,2348.33
